In [14]:
import tkinter as tk
from tkinter import *
from tkinter import filedialog
from tkinter import ttk
from ultralytics import YOLO
from PIL import Image, ImageTk , ImageGrab , ImageDraw ,ImageFont
import os
import numpy as np
import cv2
from tkinter import messagebox
# from io import BytesIO

x1=y1=x2=y2=0

def convert_to_yolo(x1, y1, x2, y2, image_width, image_height):
    w = x2 - x1
    h = y2 - y1
    x_center = x1 + (w / 2)
    y_center = y1 + (h / 2)
    x_center_norm = x_center / image_width
    y_center_norm = y_center / image_height
    w_norm = w / image_width
    h_norm = h / image_height
    # return [x_center_norm, y_center_norm, w_norm, h_norm]
    return str(x_center_norm)+" "+ str(y_center_norm) +" "+str(w_norm)+" "+ str(h_norm)

def convert_yolo_to_xyxy(x, y, w, h, image_width, image_height):
    x_center = x * image_width
    y_center = y * image_height
    width = w * image_width
    height = h * image_height

    x1 = x_center - (width / 2)
    y1 = y_center - (height / 2)
    x2 = x_center + (width / 2)
    y2 = y_center + (height / 2)
    return x1, y1, x2, y2

def convert_yolostr_to_xyxy(yolo_str, image_width, image_height):
    yolo_str=yolo_str.split()
    index=int(yolo_str[0])
    x=float(yolo_str[1])
    y=float(yolo_str[2])
    w=float(yolo_str[3])
    h=float(yolo_str[4])

    x_center = x * image_width
    y_center = y * image_height
    width = w * image_width
    height = h * image_height

    x1 = x_center - (width / 2)
    y1 = y_center - (height / 2)
    x2 = x_center + (width / 2)
    y2 = y_center + (height / 2)
    return x1, y1, x2, y2

def is_xy_in_yolo_str(yolo_str,the_x,the_y, image_width, image_height):
    yolo_str=yolo_str.split()
    index=int(yolo_str[0])
    x=float(yolo_str[1])
    y=float(yolo_str[2])
    w=float(yolo_str[3])
    h=float(yolo_str[4])

    x_center = x * image_width
    y_center = y * image_height
    width = w * image_width
    height = h * image_height

    x1 = x_center - (width / 2)
    y1 = y_center - (height / 2)
    x2 = x_center + (width / 2)
    y2 = y_center + (height / 2)
    if (x1<the_x<x2 and y1<the_y<y2) :
        return x1,y1,x2,y2, True
    else:
        return x1,y1,x2,y2, False

# def get_last_number_in_dir(directory):
#     files = os.listdir(directory)
#     numbers = [int(file.split('.')[0]) for file in files if file.endswith('.png') and file[:-4].isdigit()]
#     maximum=-1
#     if len(numbers)>0:
#         maximum = max(numbers)
#     return 

def generate_unique_filename(directory, base_filename):
    counter = 1
    while True:
        the_num=str(counter).zfill(4)
        new_filename = f"{base_filename}_{the_num}"
        if not os.path.exists(os.path.join(directory, new_filename+".png")):
            return new_filename
        counter += 1


def draw_yolo_str(yolo_str,Clear=True):
    yolo_values = yolo_str.split()
    _x1,_y1,_x2,_y2=convert_yolostr_to_xyxy ( yolo_str , base_width,base_height);
    rect_start = (min(_x1,_x2), min(_y1,_y2))
    rect_end = (max(_x1,_x2),max(_y1,_y2))
    new_image=img_resized.copy()
    draw = ImageDraw.Draw(new_image)
    draw.rectangle([rect_start, rect_end], outline="blue",width=2)
    font = ImageFont.truetype("arial.ttf", 12)
    the_label=get_index_full_text(yolo_values[0])
    # the_label=yolo_values[0]
    
    text_bbox=draw.textbbox ( (_x1,_y1-13), the_label, font=font)
    draw.rectangle(text_bbox, outline="blue",width=2,fill=(0,0,255,6)  )
    draw.text((_x1,_y1-13), the_label, fill=(255, 255, 255)  , font=font)

    img_tk = ImageTk.PhotoImage(new_image)
    canvas.create_image(0, 0, anchor='nw', image=img_tk)
    canvas.image = img_tk

def clear_canvas():
    img_tk = ImageTk.PhotoImage(img_resized)
    canvas.create_image(0, 0, anchor='nw', image=img_tk)
    canvas.image = img_tk

def on_canvas_key_press(event):
    global hover_index
    if  (event.state & 0x4 and event.char):
        key_code=ord(event.char)
        if (key_code==1):
            add_label()
        
        if (key_code==26):
            draw_all_labels()

        if (key_code==15):
            select_image()
        
        if (key_code==19):
            save_canvas_yolo()

    # if event.state & 0x4 and event.keysym == 'Delete':
    if event.keysym == 'Delete':
        result = messagebox.askyesno("Confirmation", "Are you sure you want to delete the selected label ?")
        if result:
            selected_item = lst_labels.curselection()
            if selected_item:
                lst_labels.delete(selected_item)
                clear_canvas()
                


def get_mouse_position_hover(event):
    global hover_index
    canvas.focus_set()
    if event.state & 0x1: 
        _x, _y = event.x, event.y
        hover_index=-1
        hover_index=find_object_at_xy(_x,_y)
 
def find_object_at_xy(_x,_y):
    global x1,y1,x2,y2
    for index in range(lst_labels.size()):
        yolo_str_base=lst_labels.get(index)
        yolo_str_split=yolo_str_base.split()
        _x1,_y1,_x2,_y2,is_in_yolo=is_xy_in_yolo_str(yolo_str_base,_x,_y,base_width,base_height)
        if (is_in_yolo):
            x1=min(_x1,_x2)
            x2=max(_x1,_x2)
            y1=min(_y1,_y2)
            y2=max(_y1,_y2)
            draw_yolo_str(yolo_str_base)
            lst_labels.selection_clear(0, tk.END)
            lst_labels.selection_set(index)
            lst_labels.activate(index)
            lst_labels.see(index)
            return index
        
def get_mouse_position_left(event):
    global x1,y1,x2,y2
    canvas.focus_set()
    x1 = event.x
    y1 = event.y
    lst_labels.selection_clear(0, 'end')
    change_cord_label()

def get_mouse_position_right(event):
    global x1,y1,x2,y2
    canvas.focus_set()

    lst_labels.selection_clear(0, 'end')
    x2 = event.x
    y2 = event.y
    change_cord_label()

def save_canvas_yolo():
    global yolo_base_path
    _x = window.winfo_rootx() + canvas.winfo_x()
    _y = window.winfo_rooty() + canvas.winfo_y()
    _x1 = _x + canvas.winfo_width()
    _y1 = _y + canvas.winfo_height()
    file_name = (yolo_entry_value.get())  
    save_label()
    file_path=yolo_base_path +"images/train/"+str(file_name)+".png"
    img_resized.save(file_path)
    lst_labels.delete(0, END)
    reload_files()
    lst_files_select_an_item(str(file_name)+".png",full_reload=False)

def reload_files():
    get_files('datasets/images/train')
    # yolo_entry_value.set(str(get_last_number_in_dir(yolo_base_path+"images/train")+1))
    yolo_entry_value.set(generate_unique_filename(yolo_base_path+"images/train", prefix_value.get()))


def save_label():
    global yolo_base_path
    file_path=yolo_base_path +"labels/train/"+yolo_entry_value.get()+".txt"
    labels_text=''
    for item in lst_labels.get(0, "end"):
        labels_text=labels_text+item+"\n"

    with open(file_path, 'w') as file:
        file.write(labels_text)
    
def get_index_full_text(the_index):
    the_index=int(the_index)
    for the_value in drp_object_index ['values']:
        colon_index =the_value.index(":")
        number = int(the_value[:colon_index])
        if number==the_index :
            return the_value

def add_label():
    # return (0)
    global base_width,base_height,x1,y1,x2,y2
    object_index = drp_object_index.get()[0:1]
    yolo_cord=convert_to_yolo (min(x1,x2),min(y1,y2),max(x1,x2),max(y1,y2) ,base_width,base_height)
    # lst_labels.focus_set()
    lst_labels.insert(tk.END, object_index+" "+ yolo_cord)
    lst_labels.selection_clear(0, tk.END)
    lst_labels.selection_set(lst_labels.size()-1)
    lst_labels.activate(lst_labels.size()-1)
    lst_labels.see(lst_labels.size()-1)
    lst_labels.event_generate("<<ListboxSelect>>")

def change_cord_label():
    rect_start = (min(x1,x2), min(y1,y2))
    rect_end = (max(x1,x2),max(y1,y2))
    new_image=img_resized.copy()
    draw = ImageDraw.Draw(new_image)
    draw.rectangle([rect_start, rect_end], outline="blue",width=2)
    font = ImageFont.truetype("arial.ttf", 12)
    
    the_label=drp_object_index.get()
    text_bbox=draw.textbbox ( ( rect_start[0],rect_start[1]-13), the_label, font=font)
    draw.rectangle(text_bbox, outline="blue",width=2,fill="blue")
    draw.text( ( rect_start[0],rect_start[1]-13), drp_object_index.get() , fill=(255, 255, 255)  , font=font)

    img_tk = ImageTk.PhotoImage(new_image)
    canvas.create_image(0, 0, anchor='nw', image=img_tk)
    canvas.image = img_tk

def load_file_to_canvas(file_path):
    global base_width,base_height
    global img_resized,img_base,img_child
    img_base = Image.open(file_path)
    aspect_ratio = img_base.width / img_base.height
    if base_width / base_height > aspect_ratio:
        new_width = int(base_height * aspect_ratio)
        new_height = base_height
    else:
        new_width = base_width
        new_height = int(base_width / aspect_ratio)

    img_resized = img_base.resize((new_width,new_height))
    canvas_image = Image.new("RGB", (base_width, base_height), "gray")
    x = int((base_width - new_width) / 2)
    y = int((base_height - new_height) / 2)
    
    canvas_image.paste(img_resized, (x, y))
    img_resized=canvas_image
    load_img_resized_to_canvas()
    set_canvas_child(zoom_ratio)
    lst_labels.delete(0, END)

def select_crop():
    canvas_child.place ( x=0 ,y=0)
    frame_child_buttons.place ( x=10,y=10 )

def close_crop():
    canvas_child.place ( x=640,y=640 )
    frame_child_buttons.place ( x=640,y=640 )

def crop_to_canvas():
    global img_resized,img_base,img_child
    x=-canvas_child.winfo_x()
    y=-canvas_child.winfo_y()
    img_resized = img_child.crop((x, y,x+ 640,y+ 640)) 
    load_img_resized_to_canvas()
    canvas_child.place ( x=640,y=640 )
    frame_child_buttons.place ( x=640,y=640 )

def load_img_resized_to_canvas():
    global img_resized,img_base,img_child
    img_tk = ImageTk.PhotoImage(img_resized)
    canvas.create_image(0, 0, anchor=tk.NW, image=img_tk)
    canvas.image = img_tk

def canvas_child_scroll(event):
    global zoom_ratio,rotate_deg
    if event.delta > 0:
        zoom(1.1)
    else:
        zoom(0.9)
    
def zoom(ratio,absolute=False):
        global zoom_ratio,rotate_deg
        zoom_ratio=zoom_ratio*ratio
        if absolute:zoom_ratio=ratio
        set_canvas_child(ratio=zoom_ratio ,rotate=rotate_deg)


def canvas_child_rotate( plus_deg=90):
    global zoom_ratio,rotate_deg
    rotate_deg=(rotate_deg + plus_deg ) % 360
    set_canvas_child(ratio=zoom_ratio ,rotate=rotate_deg)

def set_canvas_child(ratio=.1,rotate=0):
    global img_resized,img_base,img_child
    
    img_child=img_base.rotate(rotate,expand=True)
    
    new_width=int(img_child.width * ratio)
    new_height=int(img_child.height * ratio)
    img_child = img_child.resize((new_width,new_height))

    canvas_image = Image.new("RGB", (new_width, new_height), "gray")
    canvas_image.paste(img_child, (0, 0))
    img_child=canvas_image
    img_tk = ImageTk.PhotoImage(img_child)
    canvas_child.create_image(0, 0, anchor=tk.NW, image=img_tk)
    canvas_child.image = img_tk
    canvas_child.config(width=img_tk.width(), height=img_tk.height())

def select_image():
    close_crop()
    file_path = filedialog.askopenfilename()
    if (file_path):
        zoom_ratio=1
        rotate_deg=0
        load_file_to_canvas(file_path)        
        # window.title(version_str+"  -  "+os.path.basename(file_path))
        window.title(version_str+"  -  "+(file_path))
        yolo_entry_value.set(generate_unique_filename(yolo_base_path+"images/train", prefix_value.get()))

def delete_selected_label(event):
    
    selected_item = lst_labels.curselection()
    if selected_item and messagebox.askyesno("Confirmation", "Are you sure you want to delete the selected label ?"):
        lst_labels.delete(selected_item)

def draw_listbox_label(event):
    if len(lst_labels.curselection())==0:return
    selected_item =lst_labels.get(lst_labels.curselection()[0])
    yolo_values = selected_item.split()
    _x1,_y1,_x2,_y2=convert_yolo_to_xyxy ( float(yolo_values[1]), float(yolo_values[2]), float(yolo_values[3]) ,float(yolo_values[4]) , base_width,base_height);
    rect_start = (min(_x1,_x2), min(_y1,_y2))
    rect_end = (max(_x1,_x2),max(_y1,_y2))
    new_image=img_resized.copy()
    draw = ImageDraw.Draw(new_image)
    draw.rectangle([rect_start, rect_end], outline="blue",width=2)
    font = ImageFont.truetype("arial.ttf", 12)
    draw.text((_x1,_y1-10), yolo_values[0], fill=(255, 255, 255)  , font=font)

    img_tk = ImageTk.PhotoImage(new_image)
    canvas.create_image(0, 0, anchor='nw', image=img_tk)
    canvas.image = img_tk

def draw_all_labels():
    new_image=img_resized.copy()
    draw = ImageDraw.Draw(new_image)
    for index in range(lst_labels.size()):
        selected_item=lst_labels.get(index)
        # print (selected_item)
        yolo_values = selected_item.split()
        _x1,_y1,_x2,_y2=convert_yolo_to_xyxy ( float(yolo_values[1]), float(yolo_values[2]), float(yolo_values[3]) ,float(yolo_values[4]) , base_width,base_height);
        rect_start = (min(_x1,_x2), min(_y1,_y2))
        rect_end = (max(_x1,_x2),max(_y1,_y2))
        draw.rectangle([rect_start, rect_end], outline="blue",width=2)
        font = ImageFont.truetype("arial.ttf", 12)
        the_label=yolo_values[0]
        text_bbox=draw.textbbox ( (_x1,_y1-10), the_label, font=font)
        draw.rectangle(text_bbox, outline="blue",width=2,fill="blue")
        draw.text((_x1,_y1-10), the_label, fill=(255, 255, 255)  , font=font)


    img_tk = ImageTk.PhotoImage(new_image)
    canvas.create_image(0, 0, anchor='nw', image=img_tk)
    canvas.image = img_tk

def get_files(directory):
    file_list = os.listdir(directory)
    lst_files.delete(0, tk.END)
    for file in file_list:
        lst_files.insert(tk.END, file)

def load_yolo(event):
    if len(lst_files.curselection())==0: return
    selected_item =lst_files.get(lst_files.curselection()[0])
    load_file_to_canvas('datasets/images/train/'+selected_item)
    label_path='datasets/labels/train/'+selected_item[:-4]+".txt"
    lst_labels.delete(0, END)
    with open(label_path, 'r') as file:
        for line in file:
            lst_labels.insert(tk.END,line.rstrip())
    yolo_entry_value.set(selected_item[:-4])
    draw_all_labels()
    window.title(version_str+"  -  "+selected_item)
    
def delete_yolo(event):
    if not messagebox.askyesno("Confirmation", "Are you sure you want to delete the selected image and its labels ?"):return
    selected_item =lst_files.get(lst_files.curselection()[0])
    prev_index=lst_files.curselection()[0]
    image_path=('datasets/images/train/'+selected_item)
    label_path='datasets/labels/train/'+selected_item[:-4]+".txt"
    os.remove(image_path)
    os.remove(label_path)
    reload_files()
    lst_labels.delete(0, END)
    lst_files_select_an_index(prev_index)

def lst_files_select_an_index(the_index):
    if (the_index > lst_files.size()-1):the_index=the_index-1
    lst_files.activate(the_index)
    lst_files.selection_clear(0, tk.END)
    lst_files.selection_set(the_index)
    lst_files.activate(the_index)
    lst_files.see(the_index)
    lst_files.event_generate("<<ListboxSelect>>")

def lst_files_select_an_item(the_item,full_reload=False):
    the_index = lst_files.get(0, "end").index(the_item)
    lst_files.activate(the_index)
    lst_files.selection_clear(0, tk.END)
    lst_files.selection_set(the_index)
    lst_files.activate(the_index)
    lst_files.see(the_index)
    if (full_reload):
        lst_files.event_generate("<<ListboxSelect>>")

def auto_lable():
    global img_resized,model
    # img=np.array(img_resized)
    # img=img[:,:,::-1].copy()
    if (model==""):
        model = YOLO("best.pt")
    # model.overrides['conf'] = 0.91 # NMS confidence threshold
    # model.overrides['iou'] = 0.45  # NMS IoU threshold
    # model.overrides['agnostic_nms'] = False  # NMS class-agnostic
    # model.overrides['max_det'] = 1000  #sele maximum number of detections per image
    # image = 'datasets/images/train/2.png'
    result = model.predict(img_resized)[0]
    boxes= np.array(result.boxes.xyxy.cpu(), dtype="float")
    class_ids = np.array(result.boxes.cls.cpu(), dtype="int")
    scores = np.array(result.boxes.conf.cpu(), dtype="float")
    lst_labels.delete(0, END)
    for class_id,box in zip (class_ids,boxes):
        str_yolo=convert_to_yolo (box[0],box[1],box[2],box[3],base_width,base_height)
        str_label=str(class_id)+" "+str_yolo
        lst_labels.insert(tk.END,str_label)
    draw_all_labels()

def on_drag_start(event):
    global last_x, last_y,start_x,start_y
    last_x = event.x_root
    last_y = event.y_root
    start_x = event.x_root
    start_y = event.y_root
    
def on_drag_motion(event):
    global last_x, last_y,start_x,start_y
    delta_x = event.x_root - last_x
    delta_y = event.y_root - last_y
    cur_x=canvas_child.winfo_x()
    cur_y=canvas_child.winfo_y()
    canvas_child.place ( x= cur_x + delta_x , y=cur_y + delta_y )
    last_x = event.x_root
    last_y = event.y_root

start_x = start_y = last_x = last_y =0
hover_index=-1
model=""
dont_show_lst_lables=False
zoom_ratio=1
rotate_deg=0
base_width=640
base_height=640
img_resized=""
yolo_base_path="datasets/"
cnn_base_path="cnn_datasets/"
version_str="Aio_green_labeling by Aio green team (v1.5)"
window = tk.Tk()

window.title(version_str)

left_frame = tk.Frame(window, width=680, height=680,borderwidth=2, relief=tk.SOLID)
left_frame.grid(row=0, column=0, padx=10, pady=10)

right_frame = tk.Frame(window , width=150, height=680)
right_frame.grid(row=0, column=1, padx=10, pady=10)

canvas = tk.Canvas(left_frame, width=base_width, height=base_height)
canvas.pack()
canvas.bind("<Button-1>", get_mouse_position_left)
canvas.bind("<Button-3>", get_mouse_position_right)
canvas.bind("<Motion>", get_mouse_position_hover)

button_select = tk.Button(right_frame, text="Select Image ( Ctrl + O )", command=select_image)
button_select.pack(fill='x', expand=True)

button_select_crop = tk.Button(right_frame, text="Crop / Zoom / Rotate", command=select_crop)
button_select_crop.pack(fill='x', expand=True)

button_add = tk.Button(right_frame, text="Add label ( Ctrl + A )", command=add_label)
button_add.pack(fill='x', expand=True)

button_save_yolo = tk.Button(right_frame, text="Save Yolo ( Ctrl + S )", command=save_canvas_yolo)
button_save_yolo.pack(fill='x', expand=True)

button_auto_generate = tk.Button(right_frame, text="Autolabel", command=auto_lable)
button_auto_generate.pack(fill='x', expand=True)

lbl_cur_lable = Label(right_frame, text="Current label")
lbl_cur_lable.pack(fill='x', expand=True,pady=(10,5 ))

drp_object_index = ttk.Combobox(right_frame, values=["0 : barg"
                                                     , "1 : felfel_sabz"
                                                     , "2 : felfel_zard"
                                                     , "3 : felfel_red"
                                                     , "4 : dis_fumajin"
                                                     , "5 : dis_sefid_podri"
                                                     ])
drp_object_index.set("0 : barg")
drp_object_index.pack(fill='x', expand=True)
drp_object_index.state(['readonly'])


frame_naming=tk.Frame(right_frame)
frame_naming.pack(fill='x', expand=True,pady=(10,1))

prefix_value = StringVar()
prefix_value.set("sefid_poodri")
txt_prefix=tk.Entry(frame_naming,textvariable=prefix_value)
txt_prefix.grid(row=1, column=0, sticky="ew",padx=(0,10))

lbl_yolo_num = Label(frame_naming, text="File name", anchor='w')
lbl_yolo_num.grid(row=0, column=1, sticky="ew")

lbl_yolo_prefix = Label(frame_naming, text="File prefix", anchor='w')
lbl_yolo_prefix.grid(row=0, column=0, sticky="ew")

yolo_entry_value = StringVar()
yolo_entry_value.set(generate_unique_filename(yolo_base_path+"images/train", prefix_value.get()))
yolo_entry = tk.Entry(frame_naming, textvariable=yolo_entry_value)
yolo_entry.grid(row=1, column=1, sticky="ew")
frame_naming.grid_columnconfigure(1, weight=8)  
frame_naming.grid_columnconfigure(0, weight=2)


lbl_labels = Label(right_frame, text="Current labels")
lbl_labels.pack(fill='x', expand=True,pady=(10,0 ))

# lst_labels = tk.Listbox(right_frame,height=100)
# lst_labels.pack(fill='x', expand=True,pady=(10,1 ))
# lst_labels.bind("<Delete>", delete_selected_label)
# lst_labels.bind("<<ListboxSelect>>", draw_listbox_label)

# lst_labels_scrollbar = tk.Scrollbar(lst_labels, orient=tk.VERTICAL)
# lst_labels.config( yscrollcommand=lst_labels_scrollbar.set ,height=100)
# lst_labels_scrollbar.pack(side=tk.RIGHT, fill=tk.Y )
# lst_labels.config( height=100)


frame_labels=tk.Frame(right_frame)
frame_labels.pack(fill='both', expand=True, pady=(10, 1))
lst_labels = tk.Listbox(frame_labels, height=10)  # Adjust the height as needed
lst_labels.bind("<Delete>", delete_selected_label)
lst_labels.bind("<<ListboxSelect>>", draw_listbox_label)

lst_labels_scrollbar = tk.Scrollbar(frame_labels, orient=tk.VERTICAL)
lst_labels.config(yscrollcommand=lst_labels_scrollbar.set)
lst_labels_scrollbar.config(command=lst_labels.yview)
lst_labels_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
lst_labels.pack(fill='both', expand=True)

button_draw_all_labels = tk.Button(right_frame, text="draw all labels  ( Ctrl + Z )", command=draw_all_labels)
button_draw_all_labels.pack(fill='x', expand=True,pady=(10,0 ))

lbl_files = Label(right_frame, text="Saved Images")
lbl_files.pack(fill='x', expand=True,pady=(10,1 ))


frame_files=tk.Frame(right_frame)
frame_files.pack(fill='both', expand=True)
lst_files = tk.Listbox(frame_files,width=50)
lst_files.bind("<<ListboxSelect>>", load_yolo)
lst_files.bind("<Delete>", delete_yolo)

lst_files_scrollbar = tk.Scrollbar(frame_files, orient=tk.VERTICAL)
lst_files.config(yscrollcommand=lst_files_scrollbar.set)
lst_files_scrollbar.config(command=lst_files.yview)
lst_files_scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
lst_files.pack(fill='x', expand=True)



start_x = start_y = last_x = last_y =0
canvas_child = tk.Canvas(canvas, width=300, height=300,borderwidth=1, relief=tk.SOLID)
canvas_child.place(x=15,y=15)
canvas_child.bind("<Button-1>", on_drag_start)
canvas_child.bind("<B1-Motion>", on_drag_motion)
canvas_child.bind('<MouseWheel>', canvas_child_scroll)

frame_child_buttons=tk.Frame(canvas,width=285,height=30,borderwidth=1, relief=tk.SOLID)
frame_child_buttons.place(x=10,y=10)
button_close_child = tk.Button(frame_child_buttons, text="x", command=close_crop,width=3)
button_crop_child = tk.Button(frame_child_buttons, text="crop", command=crop_to_canvas,width=4)
button_rotate_child_right = tk.Button(frame_child_buttons, text="turn +90",command=lambda: canvas_child_rotate(plus_deg=90),width=6)
button_rotate_child_left = tk.Button(frame_child_buttons, text="turn -90", command=lambda: canvas_child_rotate(plus_deg=-90),width=6)

button_zoom_child_in = tk.Button(frame_child_buttons, text="+",command=lambda: zoom(1.1),width=3)
button_zoom_child_reset= tk.Button(frame_child_buttons, text="0", command=lambda: zoom(1,absolute=True),width=3)
button_zoom_child_out = tk.Button(frame_child_buttons, text="-", command= lambda: zoom(.9),width=3)

canvas_child.place ( x=640,y=640 )
frame_child_buttons.place ( x=640,y=640 )
button_close_child.place (x=1,y=1)
button_crop_child.place (x=35,y=1)
button_rotate_child_right.place (x=80,y=1)
button_rotate_child_left.place (x=131,y=1)
button_zoom_child_in.place (x=190,y=1)
button_zoom_child_reset.place (x=220,y=1)
button_zoom_child_out.place (x=250,y=1)
get_files('datasets/images/train')

screen_width = window.winfo_screenwidth()
screen_height = window.winfo_screenheight()

x = (screen_width // 2) - (1024 // 2)
y = (screen_height // 2) - (768 // 2)
# window.bind("<Control-s>", lambda event: save_canvas_yolo())
# window.bind("<Control-a>", lambda event: add_label())
# window.bind("<Control-o>", lambda event: select_image())
# window.bind("<Control-z>", lambda event: draw_all_labels())

canvas.focus_set()
canvas.bind("<KeyPress>", on_canvas_key_press)

window.geometry(f"{1024}x{768}+{x}+{y}")
window.mainloop()

In [51]:
import os
folder_path = 'D:/MyProjects/ai/AIGhouse/images_datasetss/mr Rabiee/abnormal/sefid_poodri/poodri'
prefix = 'sefid_poodri_'

for index , filename in enumerate(os.listdir(folder_path)):
    if os.path.isfile(os.path.join(folder_path, filename)):
        new_filename = prefix + filename
        new_filename = prefix + str(index)+".png"
        os.rename(os.path.join(folder_path, filename), os.path.join(folder_path, new_filename))